In [ ]:
import cv2
import mediapipe as mp
import matplotlib.pyplot as plt

In [ ]:
# Initialize mediapipe pose class.
mp_pose = mp.solutions.pose

# Setup the Pose function for images - independently for the images standalone processing.
pose_image = mp_pose.Pose(static_image_mode=True, min_detection_confidence=0.8)

# Initialize mediapipe drawing class - to draw the landmarks points.
mp_drawing = mp.solutions.drawing_utils

## Body Landmark Extraction

In [ ]:
image_path = "specimen3.jpg"

In [ ]:
output = cv2.imread(image_path)
original_image = output.copy()
image_in_RGB = cv2.cvtColor(output, cv2.COLOR_RGB2BGR)
resultant = pose_image.process(image_in_RGB)
mp_drawing.draw_landmarks(image=original_image, landmark_list=resultant.pose_landmarks,
                          connections=mp_pose.POSE_CONNECTIONS,
                          landmark_drawing_spec=mp_drawing.DrawingSpec(color=(0,0,0), thickness=5, circle_radius=3),
                          connection_drawing_spec=mp_drawing.DrawingSpec(color=(255,255,255), thickness=5, circle_radius=3))
figures, axis = plt.subplots(1,2, figsize=(22,22))
axis[0].title.set_text('Original')
axis[0].imshow(output[:,:,::-1])
axis[1].title.set_text('Landmarked Image')
axis[1].imshow(original_image[:,:,::-1])


#plt.figure(figsize=[22,22])
#plt.subplot(121);plt.imshow(output[:,:,::-1]);plt.title("Input Image");plt.axis('off');
#plt.subplot(122);plt.imshow(original_image[:,:,::-1]);plt.title("Pose detected Image");plt.axis('off');

## Unscaled Measurements

In [ ]:
output = cv2.imread(image_path)
h, w = output.shape[:2]
mp_pose = mp.solutions.pose
pose = mp_pose.Pose()
keypoints = pose.process(output)
lm = keypoints.pose_landmarks
lmPose  = mp_pose.PoseLandmark

#Shoulder to Shoulder Measurements
# Left shoulder.
l_shldr_x = int(lm.landmark[lmPose.LEFT_SHOULDER].x * w)
l_shldr_y = int(lm.landmark[lmPose.LEFT_SHOULDER].y * h)
# Right shoulder.
r_shldr_x = int(lm.landmark[lmPose.RIGHT_SHOULDER].x * w)
r_shldr_y = int(lm.landmark[lmPose.RIGHT_SHOULDER].y * h)

#Right Shoulder to Wrist Measurements
# Right wrist.
r_wrist_x = int(lm.landmark[lmPose.RIGHT_WRIST].x * w)
r_wrist_y = int(lm.landmark[lmPose.RIGHT_WRIST].y * h)

#Left Shoulder to Wrist Measurements
# Left wrist.
l_wrist_x = int(lm.landmark[lmPose.LEFT_WRIST].x * w)
l_wrist_y = int(lm.landmark[lmPose.LEFT_WRIST].y * h)

#Right Shoulder to Hip Measurements
# Right Hip.
r_hip_x = int(lm.landmark[lmPose.RIGHT_HIP].x * w)
r_hip_y = int(lm.landmark[lmPose.RIGHT_HIP].y * h)

#Left Shoulder to Hip Measurements
# Left hip.
l_hip_x = int(lm.landmark[lmPose.LEFT_HIP].x * w)
l_hip_y = int(lm.landmark[lmPose.LEFT_HIP].y * h)

#Right hip to Ankle Measurements
# Right Ankle.
r_ankle_x = int(lm.landmark[lmPose.RIGHT_ANKLE].x * w)
r_ankle_y = int(lm.landmark[lmPose.RIGHT_ANKLE].y * h)

#Left hip to Ankle Measurements
# Left Ankle.
l_ankle_x = int(lm.landmark[lmPose.LEFT_ANKLE].x * w)
l_ankle_y = int(lm.landmark[lmPose.LEFT_ANKLE].y * h)

import math as m
def findDistance(x1, y1, x2, y2):
    dist = m.sqrt((x2-x1)**2+(y2-y1)**2)
    return dist
shoulder_to_shoulder = findDistance(l_shldr_x, l_shldr_y, r_shldr_x, r_shldr_y)
shoulder_to_right_wrist = findDistance(r_shldr_x, r_shldr_y, r_wrist_x, r_wrist_y)
shoulder_to_left_wrist = findDistance(l_shldr_x, l_shldr_y, l_wrist_x, l_wrist_y)
shoulder_to_right_hip = findDistance(r_shldr_x, r_shldr_y, r_hip_x, r_hip_y)
shoulder_to_left_hip = findDistance(l_shldr_x, l_shldr_y, l_hip_x, l_hip_y)
hip_to_right_ankle = findDistance(r_hip_x, r_hip_y, r_ankle_x, r_ankle_y)
hip_to_left_ankle = findDistance(l_hip_x, l_hip_y, l_ankle_x, l_ankle_y)
shoulder_to_right_ankle = findDistance(r_shldr_x, r_shldr_y, r_ankle_x, r_ankle_y)
shoulder_to_left_ankle = findDistance(l_shldr_x, l_shldr_y, l_ankle_x, l_ankle_y)

print("\n\n\t UNSCALED MEASUREMENTS (cm)")
print("Shoulder Length = ", shoulder_to_shoulder)
print("Arm Length = ", shoulder_to_right_wrist)
print("Shoulder to Hip = ", shoulder_to_right_hip)
print("Trouser Length = ", hip_to_right_ankle)
print("Full Body Length (Gown) = ", shoulder_to_right_ankle)

## Reference Image Extraction

In [ ]:
# Initialise kernel for erosion/dilation
import numpy as np
kernel = np.ones((5,5),np.uint8)

img = cv2.imread(image_path) # Read image
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB) # Convert BGR (CV2 standard to RGB for proper viewing)
gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY) # Convert RGB to Gray
erode = cv2.erode(gray, kernel, iterations = 5) # Apply erosion
dilate = cv2.dilate(erode, kernel, iterations = 3) # Apply dilation
blur = cv2.GaussianBlur(dilate, (15,15), 0) # Apply guassian blur to image (to merge small items/elements) (i.e. to smooth out the edges)
edged = cv2.Canny(blur, 130, 270) # Get edges

# Show image
#figures, axis = plt.subplots(2,2, figsize=(50,50))
#axis[0,0].title.set_text('Original')
#axis[0,0].imshow(img)
#axis[0,1].title.set_text('Gray')
#axis[0,1].imshow(gray, cmap = 'gray')
#axis[1,0].title.set_text('blurred')
#axis[1,0].imshow(blur, cmap = 'gray')
#axis[1,1].title.set_text('Edged')
#axis[1,1].imshow(edged)


In [ ]:
image_copy = gray.copy()
# Convert gray image to colored (doesn't actually return the colors to the picture but allows drawing colors onto it)
image_copy = cv2.cvtColor(image_copy, cv2.COLOR_GRAY2RGB)

# Draw contours
contours, hierarchy = cv2.findContours(edged, mode = cv2.RETR_EXTERNAL, method = cv2.CHAIN_APPROX_NONE)
cv2.drawContours(image_copy, contours, -1, (255, 0, 0), 5)

# Show image
#fig=plt.figure(figsize=(15,15))
#plt.imshow(image_copy)

# Return values
#image_copy

########################################Sqaure Extraction###########################################
for ct in contours:
    perimeter = cv2.arcLength(ct, True) # Get contour parameters
    sides = cv2.approxPolyDP(ct, 0.02 * perimeter, True)
    if len(sides) == 4:
        measures = ct

image_copy = gray.copy()
# Convert gray image to colored (doesn't actually return the colors to the picture but allows drawing colors onto it)
image_copy = cv2.cvtColor(image_copy, cv2.COLOR_GRAY2RGB)
# Draw contours
cv2.drawContours(image_copy, measures, -1, (255, 0, 0), 5)
# Show image
fig=plt.figure(figsize=(10,10))
plt.imshow(image_copy)

## Scaled Measurements

In [ ]:
measures.shape
measures_new = measures.reshape(measures.shape[0],measures.shape[2])
measures_added = measures_new.sum(1)

# Get the diff between height and width
measures_diff = np.diff(measures_new, axis = 1)

# Get the 4 corners of the contour
corner1 = measures[np.argmin(measures_added)] # Corner 1 (top left)
corner4 = measures[np.argmax(measures_added)] # Corner 4 (bottom right)
corner2 = measures[np.argmin(measures_diff)] # Corner 2 (top right)
corner3 = measures[np.argmax(measures_diff)] # Corner 3 (bottom left)

# Get number of pixels to produce 1 centimeter using width
centipixel_width = ((corner2[0][0] - corner1[0][0])/15)

# Get number of pixels to produce 1 centimeter using height
centipixel_height = ((corner3[0][1] - corner1[0][1])/16.5)

print('Reference Image Width:', round(centipixel_width,3))
print('Reference Image Height:', round(centipixel_height,3))

#Extract Measurements 
print("\n\n\t SCALED MEASUREMENTS (cm)")
print("Shoulder Length = ", shoulder_to_shoulder/centipixel_width)
print("Arm Length = ", shoulder_to_right_wrist/centipixel_width)
print("Shoulder to Hip = ", shoulder_to_right_hip/centipixel_height)
print("Trouser Length = ", hip_to_right_ankle/centipixel_height)
print("Full Body Length (Gown) = ", shoulder_to_right_ankle/centipixel_height)

#print("shoulder to shoulder = ", )
#print("shoulder to right wrist = ", )
#print("shoulder to left wrist = ", shoulder_to_left_wrist/centipixel_width)
#print("shoulder to right hip = ", )
#print("shoulder to left hip = ", shoulder_to_left_hip/centipixel_height)
#print("hip to right ankle = ", )
#print("hip to left ankle = ", hip_to_left_ankle/centipixel_height)
#print("shoulder to right ankle = ", )
#print("shoulder to left ankle = ", shoulder_to_left_ankle/centipixel_height)